<div style="text-align: left;"><img src="https://www.juliabox.org/assets/img/juliacloudlogo.png" style="margin: 0px 0px 0px 0px; padding-right: 20px;width: 80px; float: left;" title="" alt="" /></div>
<img src="http://dmkpress.com/images/cms/thumbs/a5b0aeaa3fa7d6e58d75710c18673bd7ec6d5f6d/978-5-97060-370-3_270_369__100.jpg" style="margin: 0px 0px 5px 20px; width: 100px; float: right;" title="" alt="" />
Всестороннее введение в новый язык программирования для научно-технических вычислений [Julia](http://julialang.org/) в книге Малколма Шеррингтона, Packt Publishing, июль 2015.

<h1>Осваиваем язык Julia</h1><br />

Совершенствование мастерства в области аналитики и программирования при помощи Julia в целях решения задач комплексной обработки данных
<div style="text-align: left;font-size:8pt;padding-top:10px;">Программный код Julia (v0.4.5) протестирован в Windows 8.1/10 и Linux/Lubuntu 16.4</div>
<div style="text-align: left;"><h1>Глава 4. Функциональная совместимость</h1></div>

# 3. Многозадачность

In [88]:
function fibs(n = 10)
  fib = round(Int64,zeros(n))

  fib[1] = 1
  produce(fib[1]) 
  fib[2] = 1
  produce(fib[2])

  for i = 3:n
    fib[i] = fib[i-1] + fib[i-2]
    produce(fib[i])
  end
  produce(-1)
end

fibs (generic function with 2 methods)

In [89]:
p = Task(fibs)

Task (runnable) @0x00000000827b6150

<pre>
println(consume(p)); # => 1
----------------------------
----------------------------
println(consume(p)); # => 55
println(consume(p)); # => -1
</pre>

## Параллельные операции

In [ ]:
addprocs(3); # => эквивалентно запуску: julia –p 4, равно числя_ядер_процессора - 1

In [ ]:
nprocs();    # => 4 

In [ ]:
r = remotecall(2, randn, 5)

fetch(r);   # => [0.094615,0.339905,0.422061,0.790972,0.552309 ]

In [ ]:
addprocs(3)

@everywhere fib(n) = (n < 2) ? n : (fib(n-1) + fib(n-2))

In [ ]:
@elapsed fib(40);             # => 1.311472619

In [ ]:
r = @spawn @elapsed fib(40);  # => RemoteRef(4,1,17)

In [ ]:
fetch(r);                     # => 1.272693849

## Распределенные массивы

In [10]:
addprocs(3);

@everywhere using DistributedArrays

d = drand(100000);

typeof(d)  # => DArray{Float64,1,Array{Float64.1}}

DistributedArrays.DArray{Float64,1,Array{Float64,1}}

In [2]:
fieldnames(d)  

6-element Array{Symbol,1}:
 :identity
 :dims    
 :pids    
 :indexes 
 :cuts    
 :release 

In [3]:
d.cuts 

1-element Array{Array{Int64,1},1}:
 [1,33334,66668,100001]

In [4]:
d.pids

3-element Array{Int64,1}:
 2
 3
 4

In [ ]:
length(d.chunks); # => 3

In [5]:
d.indexes  

3-element Array{Tuple{UnitRange{Int64}},1}:
 (1:33333,)     
 (33334:66667,) 
 (66668:100000,)

In [10]:
d0 = dzeros(100,200,300); d0.dims  # = > (100,200,300)

(100,200,300)

In [11]:
(r, fq) = hist(d,10) 

(0.0:0.1:1.0,[10215,10011,9871,9952,9895,10103,10037,9960,10032,9924])

In [12]:
stdm(fq, mean(fq))  

103.65219620335006

In [13]:
#addprocs(3);
#@everywhere using DistributedArrays

x = @DArray [@show x^2 for x = 1:10]

	From worker 4:	x ^ 2 = 64
	From worker 3:	x ^ 2 = 16
	From worker 3:	x ^ 2 = 25
	From worker 4:	x ^ 2 = 81
	From worker 3:	x ^ 2 = 36
	From worker 4:	x ^ 2 = 100
	From worker 3:	x ^ 2 = 49
	From worker 2:	x ^ 2 = 1
	From worker 2:	x ^ 2 = 4
	From worker 2:	x ^ 2 = 9


10-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [14]:
y = @DArray [@show i + j for i = 1:3, j = 4:6]

	From worker 3:	i + j = 6
	From worker 2:	i + j = 5
	From worker 3:	i + j = 7
	From worker 2:	i + j = 6
	From worker 2:	i + j = 7
	From worker 3:	i + j = 8
	From worker 4:	i + j = 7
	From worker 4:	i + j = 8
	From worker 4:	i + j = 9


3x3 DistributedArrays.DArray{Int64,2,Array{Int64,2}}:
 5  6  7
 6  7  8
 7  8  9

## Простая модель MapReduce

In [ ]:
addprocs(3);
d = drand(300,300);
nd = length(d.chunks)

In [ ]:
μ = reduce(+, map(fetch, Any[@spawnat p mean(localpart(d)) for p in procs(d) ]))/nd;  # => 0.001171701

In [ ]:
σ = reduce(+, map(fetch, Any[@spawnat p std(localpart(d)) for p in procs(d) ]))/nd;   # => 1.001011265